In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['CRDS_PATH'] = '/Users/lilialderson/Documents/crds_cache/'

from jwst import datamodels
from jwst.pipeline import calwebb_detector1
from crds import get_default_context
print('Using crds context={}'.format(get_default_context()))

#from exotic_jedi.stage_1 import CustomBiasStep, DestripingGroupsStep

data_chunk_names = ['/Users/lilialderson/Documents/JWST/commissioning/G395H/jw01118005001_04101_00001-seg001_nrs1']
data_dir = "/Users/lilialderson/Documents/JWST/commissioning/G395H/"

#jw01118005001_04101_00001-seg001_nrs1_uncal.fits

# Data and version config.
# data_chunk_names = ['jw01366003001_04101_00001-seg001_nrs1',
#                     'jw01366003001_04101_00001-seg002_nrs1',
#                     'jw01366003001_04101_00001-seg003_nrs1']
# data_dir = '/path/to/data/jwst/nirspec/g395h/wasp39/visit_01366'
# reduction_dir = '/path/to/reduction/jwst/nirspec/g395h/wasp39/visit_01366'
# version_dir = os.path.join(reduction_dir, 'reduction_v1')
# stage_1_dir = os.path.join(version_dir, 'stage_1')
# for _dir in [version_dir, stage_1_dir]:
#     if not os.path.exists(_dir):
#         os.mkdir(_dir)

# Instantiate STScI steps for NIRSpec stage 1.
stsci_group_scale = calwebb_detector1.group_scale_step.GroupScaleStep()
stsci_dq_init = calwebb_detector1.dq_init_step.DQInitStep()
stsci_saturation = calwebb_detector1.saturation_step.SaturationStep()
stsci_superbias = calwebb_detector1.superbias_step.SuperBiasStep()
stsci_refpix = calwebb_detector1.refpix_step.RefPixStep()
stsci_linearity = calwebb_detector1.linearity_step.LinearityStep()
stsci_dark_current = calwebb_detector1.dark_current_step.DarkCurrentStep()
stsci_jump = calwebb_detector1.jump_step.JumpStep()
stsci_ramp_fit = calwebb_detector1.ramp_fit_step.RampFitStep()
stsci_gain_scale = calwebb_detector1.gain_scale_step.GainScaleStep()

# Instantiate Custom steps for NIRSpec stage 1.
#custom_bias = CustomBiasStep()
#custom_destriping_groups = DestripingGroupsStep()

# Iterate data chunks.
for data_chunk_name in data_chunk_names:
    print('\n========= Working on {} =========\n'.format(data_chunk_name))

    # Read in chunk.
    raw_data_chunk = os.path.join(
        data_dir, 'raw', '{}_uncal.fits'.format(data_chunk_name))
    dm_raw = datamodels.RampModel(raw_data_chunk)

    # Stage 1 reduction.
    proc = stsci_group_scale.call(dm_raw)
    proc = stsci_dq_init.call(proc)
    proc = stsci_saturation.call(proc, n_pix_grow_sat=1)
    # proc = stsci_superbias.call(proc)
    # proc = custom_bias.call(proc)
    proc = stsci_refpix.call(proc, odd_even_columns=True)
    proc = stsci_linearity.call(proc)
    proc = stsci_dark_current.call(proc)
    proc = stsci_jump.call(
        proc, rejection_threshold=10.,
        flag_4_neighbors=True, min_jump_to_flag_neighbors=10.,
        skip=False)
    proc = custom_destriping_groups.call(
        proc,
        start_trace_col=606, end_trace_col=2042,
        poly_order=2, n_sigma_trace_mask=15.,
        dq_bits=[0, 1, 2, 10, 11, 13, 19],
        keep_mean_bkd_level=False,
        draw_mask=False)
    _, proc = stsci_ramp_fit.call(proc)
    stage_1_output = stsci_gain_scale.call(proc)
    stage_1_output.save(path=os.path.join(
        stage_1_dir, '{}_stage_1.fits'.format(data_chunk_name)))


2023-02-28 12:04:17,213 - stpipe.GroupScaleStep - INFO - GroupScaleStep instance created.
2023-02-28 12:04:17,219 - stpipe.DQInitStep - INFO - DQInitStep instance created.
2023-02-28 12:04:17,222 - stpipe.SaturationStep - INFO - SaturationStep instance created.
2023-02-28 12:04:17,226 - stpipe.SuperBiasStep - INFO - SuperBiasStep instance created.
2023-02-28 12:04:17,228 - stpipe.RefPixStep - INFO - RefPixStep instance created.
2023-02-28 12:04:17,231 - stpipe.LinearityStep - INFO - LinearityStep instance created.
2023-02-28 12:04:17,234 - stpipe.DarkCurrentStep - INFO - DarkCurrentStep instance created.
2023-02-28 12:04:17,238 - stpipe.JumpStep - INFO - JumpStep instance created.
2023-02-28 12:04:17,241 - stpipe.RampFitStep - INFO - RampFitStep instance created.
2023-02-28 12:04:17,243 - stpipe.GainScaleStep - INFO - GainScaleStep instance created.


Using crds context=jwst_1046.pmap

========= Working on /Users/lilialderson/Documents/JWST/commissioning/G395H/jw01118005001_04101_00001-seg001_nrs1 =========



2023-02-28 12:04:26,645 - stpipe.GroupScaleStep - INFO - GroupScaleStep instance created.
2023-02-28 12:04:27,543 - stpipe.GroupScaleStep - INFO - Step GroupScaleStep running with args (<RampModel(561, 20, 32, 2048) from jw01118005001_04101_00001-seg001_nrs1_uncal.fits>,).
2023-02-28 12:04:27,545 - stpipe.GroupScaleStep - INFO - Step GroupScaleStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}
2023-02-28 12:04:49,304 - stpipe.GroupScaleStep - INFO - NFRAMES and FRMDIVSR are equal; correction not needed
2023-02-28 12:04:49,312 - stpipe.GroupScaleStep - INFO - Step will be skipped
2023-02-28 12:04:49,320 - stpipe.GroupScaleStep - INFO - Step GroupScaleStep done
2023-02-28 12:04:49,371 - stpipe.DQInitStep - INFO - DQInitStep instance created.
2023-02-28 12:04:49,803 - st

2023-02-28 12:08:37,795 - stpipe.JumpStep - INFO - CR rejection threshold = 10 sigma
2023-02-28 12:08:37,820 - CRDS - INFO -  Fetching  /Users/lilialderson/Documents/crds_cache/references/jwst/nirspec/jwst_nirspec_gain_0025.fits    2.1 M bytes  (1 / 1 files) (0 / 2.1 M bytes)
2023-02-28 12:08:39,214 - stpipe.JumpStep - INFO - Using GAIN reference file: /Users/lilialderson/Documents/crds_cache/references/jwst/nirspec/jwst_nirspec_gain_0025.fits
2023-02-28 12:08:39,281 - CRDS - INFO -  Fetching  /Users/lilialderson/Documents/crds_cache/references/jwst/nirspec/jwst_nirspec_readnoise_0043.fits    4.2 M bytes  (1 / 1 files) (0 / 4.2 M bytes)
2023-02-28 12:08:40,778 - stpipe.JumpStep - INFO - Using READNOISE reference file: /Users/lilialderson/Documents/crds_cache/references/jwst/nirspec/jwst_nirspec_readnoise_0043.fits
2023-02-28 12:09:01,247 - stpipe.JumpStep - INFO - Extracting gain subarray to match science data
2023-02-28 12:09:01,253 - stpipe.JumpStep - INFO - Extracting readnoise suba

2023-02-28 12:09:13,116 - stpipe.JumpStep - INFO - Working on integration 15:
2023-02-28 12:09:13,216 - stpipe.JumpStep - INFO - From highest outlier, two-point found 147 pixels with at least one CR from five groups.
2023-02-28 12:09:13,217 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:09:13,218 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:09:13,337 - stpipe.JumpStep - INFO - Working on integration 16:
2023-02-28 12:09:13,441 - stpipe.JumpStep - INFO - From highest outlier, two-point found 119 pixels with at least one CR from five groups.
2023-02-28 12:09:13,442 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:09:13,443 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:09

...

2023-02-28 12:10:17,981 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:18,098 - stpipe.JumpStep - INFO - Working on integration 300:
2023-02-28 12:10:18,220 - stpipe.JumpStep - INFO - From highest outlier, two-point found 190 pixels with at least one CR from five groups.
2023-02-28 12:10:18,221 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:10:18,222 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:18,356 - stpipe.JumpStep - INFO - Working on integration 301:
2023-02-28 12:10:18,463 - stpipe.JumpStep - INFO - From highest outlier, two-point found 177 pixels with at least one CR from five groups.
2023-02-28 12:10:18,464 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:

2023-02-28 12:10:22,558 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:10:22,559 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:22,681 - stpipe.JumpStep - INFO - Working on integration 317:
2023-02-28 12:10:22,795 - stpipe.JumpStep - INFO - From highest outlier, two-point found 200 pixels with at least one CR from five groups.
2023-02-28 12:10:22,796 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:10:22,797 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:22,942 - stpipe.JumpStep - INFO - Working on integration 318:
2023-02-28 12:10:23,051 - stpipe.JumpStep - INFO - From highest outlier, two-point found 122 pixels with at least one CR from five groups.
2023-02-28 12:

2023-02-28 12:10:26,479 - stpipe.JumpStep - INFO - From highest outlier, two-point found 166 pixels with at least one CR from five groups.
2023-02-28 12:10:26,480 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:10:26,480 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:26,598 - stpipe.JumpStep - INFO - Working on integration 334:
2023-02-28 12:10:26,702 - stpipe.JumpStep - INFO - From highest outlier, two-point found 141 pixels with at least one CR from five groups.
2023-02-28 12:10:26,703 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:10:26,703 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:26,833 - stpipe.JumpStep - INFO - Working on integration 335:
2023-02-28 12:

2023-02-28 12:10:30,875 - stpipe.JumpStep - INFO - Working on integration 350:
2023-02-28 12:10:30,987 - stpipe.JumpStep - INFO - From highest outlier, two-point found 172 pixels with at least one CR from five groups.
2023-02-28 12:10:30,988 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:10:30,989 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:31,140 - stpipe.JumpStep - INFO - Working on integration 351:
2023-02-28 12:10:31,254 - stpipe.JumpStep - INFO - From highest outlier, two-point found 151 pixels with at least one CR from five groups.
2023-02-28 12:10:31,255 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:10:31,256 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:

2023-02-28 12:10:35,662 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:35,790 - stpipe.JumpStep - INFO - Working on integration 367:
2023-02-28 12:10:35,901 - stpipe.JumpStep - INFO - From highest outlier, two-point found 184 pixels with at least one CR from five groups.
2023-02-28 12:10:35,902 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12:10:35,902 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from three groups.
2023-02-28 12:10:36,059 - stpipe.JumpStep - INFO - Working on integration 368:
2023-02-28 12:10:36,168 - stpipe.JumpStep - INFO - From highest outlier, two-point found 142 pixels with at least one CR from five groups.
2023-02-28 12:10:36,169 - stpipe.JumpStep - INFO - From highest outlier, two-point found 0 pixels with at least one CR from four groups.
2023-02-28 12: